In [1]:
%%html
<h1 align=center> Bem vindos ao Caderno de Análises do EFD ICMS <br>
<img src="qrcode.png" width="200px">
<h2 align="center"> <a href="http://jupyter.org" target="_blank"> Powered By Jupyter Notebooks </a>

In [ ]:
# Importar as bibliotecas para a realização da Análise ((não alterar o código))

from sqlalchemy import create_engine
import pymysql
import datetime as dt
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Leitura de dados da tabela cfop
cfop_df = pd.read_excel("tabela_cfop_2019.xlsx", skiprows=2)[["CFOP", "Descrição Resumida"]]
cfop_df.rename(columns={"CFOP":"CFOP", "Descrição Resumida":"DESCRICAO"}, inplace=True)

In [ ]:
# Conexão ao banco de dados do EFD-Cotribuições (não alterar o código)
conn = pymysql.connect(host="localhost",
                      port=3337,
                      user="spedfiscal",
                      passwd="spedfiscal",
                      database = "master",
                      charset='utf8')

In [ ]:
%%html
<h2 align="center"> Insira o NOME da entidade sem espaços, simbolos ou letras </h2>

In [ ]:
name = input("Digite o NOME da entidade sem símbolos: ")
initial_data = input("Digite a data inicial no formato: yyyy-mm-day")

In [ ]:
%%html
<h4 align="center"> Copie na opção abaixo o nome do db para análise </h4>

In [ ]:
# Visualizando os nomes dos Bancos de Dados , entidades e CNPJ
escrituracao_bds = pd.read_sql(f"select cpf_cnpj, nome_contribuinte, datainicial, uf,nomeBD FROM escrituracaofiscal WHERE nome_contribuinte LIKE '%{name}%' AND datainicial = '{initial_data}'" , con=conn)
conn.close()
data_bases = escrituracao_bds.nomeBD
escrituracao_bds

In [ ]:
# Função de conexão ao banco de dados para iteração de cada BD
def connection(data_base):
    conn = pymysql.connect(host="localhost",
                      port=3337,
                      user="spedfiscal",
                      passwd="spedfiscal",
                      database = f"{data_base}",
                      charset='utf8')
    return conn

In [ ]:
%%html
<h2> Tabela de entrada e saída de mercadorias para todos as UFs - reg_c100 </h2>

In [ ]:
# Create the first Data Frame reg_c100
conn = pymysql.connect(host="localhost",
                      port=3337,
                      user="spedfiscal",
                      passwd="spedfiscal",
                      database = data_bases[0],
                      charset='utf8')
reg_c100_all = pd.read_sql("SELECT COD_PART, NUM_DOC, CHV_NFE, DT_DOC, VL_DOC FROM reg_c100 WHERE IND_OPER = 1", conn)
conn.close()    

In [ ]:
# append anothers to DF
for i in range(1,data_bases.size):
    conn = connection(data_bases[i])
    reg_c100_all = reg_c100_all.append(pd.read_sql("SELECT COD_PART, NUM_DOC, CHV_NFE, DT_DOC, VL_DOC FROM reg_c100 WHERE IND_OPER = 1", conn))
    conn.close()

In [ ]:
# Registros de saída de mercadorias e Notas Fiscais
reg_c100_all

In [ ]:
%%html
<h2> Registros de Mercadorias C190 por CFOP </h2>

In [ ]:
# Create the first Data Frame reg_c190
conn = pymysql.connect(host="localhost",
                      port=3337,
                      user="spedfiscal",
                      passwd="spedfiscal",
                      database = data_bases[0],
                      charset='utf8')
reg_c190_all = pd.read_sql("SELECT ID_PAI, CFOP, VL_OPR FROM reg_c190 WHERE CFOP >= 5000", con=conn)
conn.close()

In [ ]:
# append anothers to DF
for i in range(1,data_bases.size):
    conn = connection(data_bases[i])
    reg_c190 = reg_c190_all.append(pd.read_sql("SELECT ID_PAI, CFOP, VL_OPR FROM reg_c190 WHERE CFOP >= 5000", con=conn))
    conn.close()

In [ ]:
reg_c190 = reg_c190.astype({"CFOP" : "int64"})
reg_c190_cfop_merge = pd.merge(reg_c190, cfop_df, on="CFOP").sort_values(by="ID_PAI")
reg_c190_cfop_merge

In [ ]:
# AGREGAÇÃO DE VALORES (SOMA) POR CFOP
reg_c190_group_sum = reg_c190_cfop_merge[["DESCRICAO","CFOP" , "VL_OPR"]].groupby(["CFOP","DESCRICAO"]).agg("sum")
reg_c190_group_sum

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Registros d190 - Saída de Produtos e Mercadorias por código de CFOP")
plt.ylabel("Classificação das atividades")
plt.xlabel("Valores dos itens")
sns.histplot(data=reg_c190_group_sum, x="VL_OPR", hue="DESCRICAO")
plt.show()

In [ ]:
%%html
<h2> Análise da Prestação de Serviços de telecomunicação D696 - saída </h2>

In [ ]:
# Create the first Data Frame reg_d696
conn = pymysql.connect(host="localhost",
                      port=3337,
                      user="spedfiscal",
                      passwd="spedfiscal",
                      database = data_bases[0],
                      charset='utf8')
reg_d696 = pd.read_sql("SELECT HASH, CFOP, VL_OPR FROM reg_d696", conn)
conn.close()

In [ ]:
# append anothers to DF
for i in range(1,data_bases.size):
    conn = connection(data_bases[i])
    reg_d696 = reg_d696.append(pd.read_sql("SELECT HASH, CFOP, VL_OPR FROM reg_d696", conn))
    conn.close()

In [ ]:
reg_d696

In [ ]:
# VALORES CLASSIFICADOS POR CFOP
reg_d696 = reg_d696.astype({"CFOP" : "int64"})
reg_d696_telecom_cfop_merge = pd.merge(reg_d696, cfop_df, on="CFOP")
reg_d696_telecom_cfop_merge

In [ ]:
## Soma de todas as Notas Fiscais de saídas do Registro d5696:
print(f" Valor total das NFs de saída telecom: R$ {round(reg_d696_telecom_cfop_merge.VL_OPR.sum(),2)}")

In [ ]:
# Soma de valores por código de itens
reg_d696_agg_by_cod_item = reg_d696_telecom_cfop_merge[["DESCRICAO","VL_OPR"]].groupby(["DESCRICAO"]).agg(["sum", "mean","min", "max"])
reg_d696_agg_by_cod_item

In [ ]:
%%html
<h2 align="center"> Algumas Análises Gráficas </h2>

In [ ]:
#plt.figure(figsize=(12,8))
plt.title("Registros d696 - Serviços de telecomunicações por tipo de atividade")
plt.ylabel("Classificação das atividades")
plt.xlabel("Valores dos itens")
sns.barplot(data=reg_d696_telecom_cfop_merge, y="DESCRICAO", x="VL_OPR")
plt.show()

In [ ]:
# plt.figure(figsize=(12,8))
plt.title("Box plot das atividades por serviços de telecomunicações D696")
sns.boxplot(data=reg_d696_telecom_cfop_merge, y="DESCRICAO", x="VL_OPR")
plt.show()

In [ ]:
%%html
<h2> Análise da Prestação de Serviços de telecomunicação D500 - saída </h2>

In [ ]:
conn = connection(data_bases[0])

In [ ]:
reg_d510_telecom = pd.read_sql("SELECT HASH, COD_ITEM, VL_ITEM, CFOP, ALIQ_ICMS, COD_CTA FROM reg_d510", con=conn)
reg_d510_telecom

In [ ]:
## Soma de todas as Notas Fiscais de saídas do Registro d500:
print(f" Valor total das NFs de saída telecom: R$ {round(reg_d510_telecom.VL_ITEM.sum(),2)}")


In [ ]:
# Soma de valores por código de itens
reg_d510_agg_by_cod_item = reg_d510_telecom[["COD_ITEM","VL_ITEM"]].groupby(["COD_ITEM"]).agg(["sum", "mean","min", "max"])
reg_d510_agg_by_cod_item

In [ ]:
%%html
<h2 align="center"> Soma de todos os itens de telecom </h2>

In [ ]:
# soma de todos das receitas de todos os Serviços de telecom:
total_d510 = reg_d510_agg_by_cod_item["VL_ITEM"]["sum"].sum()
print(f"valor total dos Serviços : R$ {round(total_d510,2)}")

In [ ]:
%%html
<h2 align="center"> Algumas Análises Gráficas </h2>


In [ ]:
plt.figure(figsize=(12,8))
plt.title("Registros d510 - Serviços de telecomunicações por tipo de atividade")
plt.ylabel("Classificação das atividades")
plt.xlabel("Valores dos itens")
sns.barplot(data=reg_d510_telecom, y="COD_ITEM", x="VL_ITEM", hue="COD_CTA")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.title("Box plot das atividades por serviços de telecomunicações D500")
sns.boxplot(data=reg_d510_telecom, y="COD_ITEM", x="VL_ITEM", hue="COD_CTA")
plt.show()

In [ ]:
conn.close()

In [ ]:
%%html
<h3 align="center"> Execute a célula abaixo Se e somente Se desejar gravar as tabelas no formato excell </h3>

In [ ]:
# Importing files to Excell
month = input("Escreva o nome do mês em minúsculo, sem separações ou símbolos: ")
# reg_d510_telecom_merged.to_excel(f"servicos_telecom_d510_{month}.xlsx")
# reg_d510_agg_by_cod_item.to_excel(f"soma_agregada_telecom_d510_{month}.xlsx")
reg_c190_cfop_merge.to_excel(f"saidas_mercadorias_cfop_c190_{month}.xlsx")
reg_c190_group_sum.to_excel(f"saidas_mercadorias_agg_c190_{month}.xlsx")
reg_c100_all.to_excel(f"notas_fiscais_c100{month}.xlsx")
reg_d696_agg_by_cod_item.to_excel(f"reg_telecom_grouped_d696_{month}.xlsx")
reg_d696_telecom_cfop_merge.to_excel(f"reg_telecom_cfop_d696_{month}.xlsx")
